In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [3]:
class PieChartCreator:
    def __init__(self, df, building_no, zone_name, date_range=None):
        self._df = df.copy()
        self._building_no = building_no
        self._date_range = date_range
        self._zone_name = zone_name    
        self._fig = self.update_fig(df, building_no, zone_name, date_range)
        
        
    @property
    def df(self):
        return self._df
    
    @property
    def building_no(self):
        return self._building_no
    
    @property
    def date_range(self):
        return self._date_range
    
    @property
    def zone_name(self):
        return self._zone_name
    
    @property
    def fig(self):
        return self._fig
    
    def update_fig(self, df, building_no, zone_name, date_range=None):
        # Suppress Plotly Deprecated dt warning
        import warnings
        warnings.simplefilter("ignore", category=FutureWarning)

        # Subset data
        df = df[(df.building_no==building_no)&(df.Fan_status=='On')&(df.Zone_name==zone_name)]
        if date_range is not None:
            temp = pd.Series(pd.to_datetime(df.Datetime, errors='coerce'), index=df.index, name='Datetime')
            df = df[(temp.dt.date>=date_range[0])&(temp.dt.date<date_range[1])]

        fig = go.Figure()
            
        # Add main plots
        fig.add_trace(
            go.Pie(
                labels=['Savings','Predicted Fan Usage'],
                values=[df.Fan_time_diff.sum()-df.Predicted.sum(),df.Predicted.sum()],
                hole=0.2,
                pull=[0, 0.1]
            )
        )
        
        return fig

In [4]:
df = pd.read_parquet(r'summary_all.parquet')

In [5]:
display(PieChartCreator(df, 1, 'Acu101').fig)

In [6]:
display(PieChartCreator(df,1,'Acu101').update_fig(df, 1, 'Acu304'))

In [61]:
import datetime
display(PieChartCreator(df,1,'Acu101').update_fig(df, 1, 'Acu101', [datetime.date(2023, 7, 4), datetime.date(2023, 7, 5)]))